In [57]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [58]:
h_0 = 0.01
l_x = 1
l_y = np.sqrt(2)

n_x = int(np.round(l_x / h_0))
n_y = int(np.round(l_y / h_0))

h_x = l_x / n_x
h_y = l_y / n_y

x = np.linspace(h_x / 2, l_x - h_x / 2, n_x)
y = np.linspace(h_y / 2, l_y - h_y / 2, n_y)

X, Y = np.meshgrid(x, y, indexing='ij')


In [59]:
def f(x, y):
  return np.sin(5 * x) * ((25 + np.pi ** 2 * y ** 2) * np.cos(np.pi * y ** 2 / 2) + np.pi * np.sin(np.pi * y ** 2 / 2))


def μ_L(y):
  return 0


def μ_R(y):
  return np.sin(5) * np.cos(np.pi * y ** 2 / 2)


def μ_B(x):
  return np.sin(5 * x)


def μ_T(x):
  return -np.sin(5 * x)


def u_0(x, y):
  return np.sin(5 * x) * np.cos(np.pi * y ** 2 / 2)

In [60]:
def F(X, Y, h_x, h_y):
    res = f(X, Y)

    res[0, :] += 2 * μ_L(Y[0, :]) / h_x ** 2

    res[-1, :] += 2 * μ_R(Y[-1, :]) / h_x ** 2

    res[:, 0] += 2 * μ_B(X[:, 0]) / h_y ** 2

    res[:, -1] += 2 * μ_T(X[:, -1]) / h_y ** 2

    return res


def L_tilda(u, h_x, h_y):
    res = np.zeros_like(u)

    res[1:, :] -= u[:-1, :] / h_x ** 2

    res[:-1, :] -= u[1:, :] / h_x ** 2

    res[:, 1:] -= u[:, :-1] / h_y ** 2

    
    res[:, :-1] -= u[:, 1:] / h_y ** 2

    return res




def D(u, h_x, h_y):
    res = np.zeros_like(u)

    res[1:, :] += 1 / h_x ** 2

    res[0, :] += 2 / h_x ** 2

    res[:-1, :] += 1 / h_x ** 2

    res[-1, :] += 2 / h_x ** 2

    res[:, 1:] += 1 / h_y ** 2

    res[:, 0] += 2 / h_y ** 2

    res[:, :-1] += 1 / h_y ** 2

    res[:, -1] += 2 / h_y ** 2

    return res


In [61]:
def F_i(X, Y, h_x, h_y, i):
    res = f(X[i], Y[i])
    if (i == 0):
        res += 2 * μ_L(Y[0, :]) / h_x ** 2
    if (i == X.shape[0]-1):
        res += 2 * μ_R(Y[-1, :]) / h_x ** 2

    res[0] += 2 * μ_B(X[i, 0]) / h_y ** 2

    res[-1] += 2 * μ_T(X[i, -1]) / h_y ** 2

    return res


def L_tilda_i(u, h_x, h_y,i):
    res = np.zeros_like(u[i])
    if (i>0):
        res -= u[i-1] / h_x ** 2
    if (i<u.shape[0]-1):
        res -= u[i+1] / h_x ** 2

    res[1:] -= u[i,:-1] / h_y ** 2

    
    res[:-1] -= u[i,1:] / h_y ** 2

    return res

def D_i(u, h_x, h_y, i):
    res = np.zeros_like(u[i])
    if (i>0):
        res += 1 / h_x ** 2
    if (i==0):
        res += 2 / h_x ** 2

    if (i<u.shape[0]-1):
        res += 1 / h_x ** 2

    if (i == u.shape[0]-1):
        res += 2 / h_x ** 2
    
    res[1:] += 1 / h_y ** 2

    res[0] += 2 / h_y ** 2

    res[:-1] += 1 / h_y ** 2

    res[-1] += 2 / h_y ** 2

    return res

In [62]:
def Jacobi_method_chopped(u, h_x, h_y, i):
    
    D_val = D_i(u, h_x, h_y, i)
    

    
    u = 1 / D_val * (F_i(X, Y, h_x, h_y,i)/2 - L_tilda_i(u, h_x, h_y,i))
        
    return u





def Zeidel(h_x,h_y,ε,max_iter,error = None):
    u = np.zeros_like(X)
    

    for _ in range(max_iter):
        
        u_k1 = np.zeros_like(X)
        for i in range(X.shape[0]):
            u_k = np.copy(u)
            for j in range(i):
                u_k[j] = np.zeros(X.shape[1])
            u_k1[i] = Jacobi_method_chopped(u_k1,h_x,h_y, i) + Jacobi_method_chopped(u_k,h_x,h_y, i)
        u = u_k1
    return u


In [63]:
ε = 1e-2
max_iter = 10000

u_exact = u_0(X, Y)
u_num = Zeidel(h_x, h_y, ε, max_iter)

In [64]:
fig = make_subplots(rows=1, cols=2, subplot_titles=["Numerical solution", "Analytical solution"])

fig.add_trace(go.Heatmap(z=u_num.T, colorscale='Magma', x0=0, dx=l_x, y0=0, dy=l_y, colorbar=dict(title='u')), row=1, col=1)
fig.add_trace(go.Heatmap(z=u_exact.T, colorscale='Magma', x0=0, dx=l_x, y0=0, dy=l_y, colorbar=dict(title='u')), row=1, col=2)

fig.update_layout(
    width=1000,
    height=500,
)

fig.update_xaxes(title_text='x', row=1, col=1)
fig.update_xaxes(title_text='x', row=1, col=2)
fig.update_yaxes(title_text='y', row=1, col=1)
fig.update_yaxes(title_text='y', row=1, col=2)

fig.show()

**Вычисление погрешностей и указание порядка аппроксимации**

In [65]:
h_0 = 0.1

h = []
fault_array = []

for i in range(4):
    n_x = int(np.round(l_x / h_0))
    n_y = int(np.round(l_y / h_0))

    h_x = l_x / n_x
    h_y = l_y / n_y

    x = np.linspace(h_x / 2, l_x - h_x / 2, n_x)
    y = np.linspace(h_y / 2, l_y - h_y / 2, n_y)

    X, Y = np.meshgrid(x, y, indexing='ij')

    u_exact = u_0(X, Y)
    u_num = Zeidel(h_x, h_y, ε, max_iter)

    fault_array.append(np.max(np.abs(u_exact - u_num)))

    h.append(h_0)

    h_0 /= 2

In [66]:
tg = abs(np.log(fault_array[0]) - np.log(fault_array[3])) / abs(np.log(h[0]) - np.log(h[3]))
print(tg)

fig = go.Figure(data=[go.Scatter(x=h, y=fault_array, mode='lines+markers', line=dict(color='black'), marker=dict(color='black'))])

fig.update_layout(title='Evaluation of the accuracy of the method',
                  xaxis_title='h',
                  yaxis_title='max_err',
                  xaxis=dict(type='log'),
                  yaxis=dict(type='log'))

fig.show()

1.9820429886783397
